# Redes Bayesianas

Supongamos una situación en la que tenemos tres variables: **Caries**, **Dolor** y **Huecos**. Por nuestro propio conocimiento, podemos inferir que la caries es la causante del dolor y/o de los huecos en los dientes. Por ese motivo representaremos la red bayesiana con tres nodos, donde el nodo **Caries** es padre de los nodos **Dolor** y **Huecos**. Representaremos en sus respectivas tablas sus valores probabilísticos.

<img src="imgs/red_bayesiana.JPG" width=70%>

Utilizaremos la librería *pomegranate* para crear la red y hacer inferencia sobre ella.

In [23]:
%pylab inline
from pomegranate import *

caries = DiscreteDistribution({'Yes': 0.8, 'No': 0.2})

dolor = ConditionalProbabilityTable(
        [['Yes', 'True', 0.6],
         ['Yes', 'False', 0.4],
         ['No', 'True', 0.1],
         ['No', 'False', 0.9]],
         [caries])

huecos = ConditionalProbabilityTable(
        [['Yes', 'True', 0.9],
         ['Yes', 'False', 0.1],
         ['No', 'True', 0.2],
         ['No', 'False', 0.8]],
         [caries])

s1 = Node(caries, name="caries")
s2 = Node(dolor, name="dolor")
s3 = Node(huecos, name="huecos")

model = BayesianNetwork("Dentista")
model.add_states(s1, s2, s3)
model.add_edge(s1, s2)
model.add_edge(s1, s3)

model.bake()
# model.plot()

Populating the interactive namespace from numpy and matplotlib


Calcularemos ahora qué probabilidad de padecer caries tenemos si nos duele una muela.

In [24]:
observations = { 'dolor' : 'True'}

beliefs = model.predict_proba(observations)

for state, belief in zip( model.states, beliefs ):
    if type(belief) is not str:
        print(state.name)
        print("Estado", belief.parameters)

caries
Estado [{'Yes': 0.9599999999999996, 'No': 0.04000000000000032}]
huecos
Estado [{'True': 0.8719999999999998, 'False': 0.12800000000000028}]


#### Ejercicio 1

Calcula a mano y mediante la librería los siguientes casos:

- Probabilidad de padecer caries tenemos si tenemos huecos.
- Probabilidad de padecer caries.
- Probabilidad de no padecer caries tenemos si tenemos huecos y dolor.
- Probabilidad de tener huecos.

Verifica que lo que calculas a mano coincide con las respuestas de la librería.

#### Ejercicio 2

Implementa mediante la librería la siguiente red:

<img src="imgs/red_bayesiana_ejemplo_2.JPG" width=70%>

Calcula los valores de las diferentes variables si se produce un robo.

In [19]:
%pylab inline
from pomegranate import *

robo = DiscreteDistribution({'Yes_robo': 0.001, 'No_robo': 0.999})
terremoto = DiscreteDistribution({'Yes_terremoto': 0.002, 'No_terremoto': 0.998})

alarma = ConditionalProbabilityTable(
        [['Yes_robo', 'Yes_terremoto', 'Salta_alarma', 0.95],
         ['Yes_robo', 'Yes_terremoto', 'No_salta_alarma', 0.05],
         ['Yes_robo', 'No_terremoto', 'Salta_alarma', 0.94],
         ['Yes_robo', 'No_terremoto', 'No_salta_alarma', 0.06],
         ['No_robo', 'Yes_terremoto', 'Salta_alarma', 0.29],
         ['No_robo', 'Yes_terremoto', 'No_salta_alarma', 0.71],
         ['No_robo', 'No_terremoto', 'Salta_alarma', 0.001],
         ['No_robo', 'No_terremoto', 'No_salta_alarma', 0.999]],
         [robo, terremoto])

juan = ConditionalProbabilityTable(
        [['Salta_alarma','Juan_llama', 0.90],
         ['No_salta_alarma','Juan_llama', 0.10],
         ['Salta_alarma','Juan_no_llama', 0.05],
         ['No_salta_alarma','Juan_no_llama', 0.95]],
         [alarma])

maria = ConditionalProbabilityTable(
        [['Salta_alarma','Maria_llama', 0.70],
         ['No_salta_alarma','Maria_llama', 0.30],
         ['Salta_alarma','Maria_no_llama', 0.01],
         ['No_salta_alarma','Maria_no_llama', 0.99]],
         [alarma])

s1 = Node(robo, name="Robo")
s2 = Node(terremoto, name="Terremoto")
s3 = Node(alarma, name="Alarma")
s4 = Node(juan, name="Juan")
s5 = Node(maria, name="Maria")


model = BayesianNetwork("Avisos")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.add_edge(s3, s4)
model.add_edge(s3, s5)

model.bake()
#model.plot()

Populating the interactive namespace from numpy and matplotlib


In [22]:
observations = {'Robo':'True'}

beliefs = model.predict_proba(observations)

for state, belief in zip( model.states, beliefs ):
    if type(belief) is not str:
        print(state.name)
        print("Estado", belief.parameters)

Terremoto
Estado [{'Yes_terremoto': 0.0020000000000004424, 'No_terremoto': 0.9979999999999996}]
Alarma
Estado [{'No_salta_alarma': 0.6675702316412025, 'Salta_alarma': 0.3324297683587975}]
Juan
Estado [{'Juan_no_llama': 0.6214884179398719, 'Juan_llama': 0.37851158206012825}]
Maria
Estado [{'Maria_no_llama': 0.5170034499753573, 'Maria_llama': 0.4829965500246427}]
